In [10]:
import json
import os
from tqdm import tqdm

model_name = ['HC3']
method = 'nlp'

for name in model_name:
    result = []
    watermark_dataset = json.load(open(f'../nlp-watermarking-main/result/{method}/HC3/watermark/{name}-random-800.json'))
    original_dataset = json.load(open(f'dataset-random-800.json'))
    result_file = f'text-quality/{method}/HC3/{name}.json'
    print(f'======\nresult => {result_file}')
    for index in tqdm(range(len(watermark_dataset))):
        original_text = original_dataset[index]
        watermark_text = ''
        for j in range(len(watermark_dataset[index])):
            watermark_text += watermark_dataset[index][j]['clean_wm_text']
        result.append({
            'original_text': original_text,
            'watermark_text':watermark_text
        })
    os.makedirs(os.path.dirname(result_file), exist_ok=True)
    with open(result_file, 'w', encoding='utf-8') as f:
        json.dump(result, f, indent=4, ensure_ascii=False)


result => text-quality/nlp/HC3/HC3.json


100%|██████████| 800/800 [00:00<00:00, 231841.58it/s]


In [4]:
from watermark_REPEAT_NO_CONTEXT_BERT import watermark_model as repeat_no_context_model
model = repeat_no_context_model()

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
/root/miniconda3/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:818: UserWarning: `return_dict_in_generate` is NOT set to `True`, but `

===bert-base-cased 完毕===


Some weights of the model checkpoint at /root/autodl-tmp/roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


===roberta-large-mnli 完毕===
===roberta-large完毕===
===glove 完毕===
===nltk完毕===
===transform_model完毕===
===mapping完毕===
===cuda===
===REPEAT_NO_CONTEXT===
===models/transform_model-bert_large-DISTANCE-CONDITION.pth===


/root/TextRepeat/watermark_REPEAT_NO_CONTEXT_BERT.py:144: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  transform_model.load_state_dict(torch.load(transform_model_path, map_

In [5]:
import json
import os
from utils1 import bias
from tqdm import tqdm

model_name = ['HC3']
method = 'REPEAT'
dataset_name = 'HC3'
result = []

for name in model_name:
    file_name = f'text-quality/{method}/{dataset_name}/{name}.json'
    dataset = json.load(open(file_name))
    sim_total = 0
    sim_95 = 0
    sim_9 = 0
    print(f'==={name}===')
    for index in tqdm(range(len(dataset))):
        original_embedding = bias.get_embedding(model.device, model.embedding_model, model.embedding_tokenizer, dataset[index]['original_text'])
        watermark_embedding = bias.get_embedding(model.device, model.embedding_model, model.embedding_tokenizer, dataset[index]['watermark_text'])
        sim = bias.cosine_similarity(original_embedding, watermark_embedding)
        if sim.item()>0.95:
            sim_95 += 1
        if 0.9 < sim.item() < 0.95:
            sim_9 += 1
        sim_total += sim.item()
    average_sim = sim_total / len(dataset)
    result.append({
        'name': name,
        'sim95':sim_95,
        'sim9': sim_9,
        'average': average_sim
    })
    os.makedirs(os.path.dirname(f'text-quality/{method}/{dataset_name}/result.json'), exist_ok=True)
    with open(f'text-quality/{method}/{dataset_name}/result.json', 'w', encoding='utf-8') as f:
        json.dump(result, f, indent=4, ensure_ascii=False)
    # print(f'{name} =>  {sim_total} / {len(dataset)} === {average_sim}')

===HC3===


100%|██████████| 800/800 [00:14<00:00, 55.09it/s]


In [4]:
import json
import os
from tqdm import tqdm
from utils1 import bias
from watermark_REPEAT_NO_CONTEXT_BERT import watermark_model as repeat_no_context_model

model = repeat_no_context_model()

model_name = ['bloomz', 'chatGPT', 'cohere', 'davinci', 'dolly', 'flant5']

result = []
for name in model_name:
    # file_name = f'../result/FASTER/water/arxiv/abstract/m4-{name}-0.75-800.json'
    file_name = f'../result/REPEAT/water/arxiv/abstract/m4-{name}-0.75-800.json'
    dataset = json.load(open(file_name))
    sim_total = 0
    sim_95 = 0
    sim_9 = 0
    print(f'==={name}===')
    for index in tqdm(range(len(dataset))):
        original_embedding = bias.get_embedding(model.device, model.embedding_model, model.embedding_tokenizer, dataset[index]['original_text'])
        watermark_embedding = bias.get_embedding(model.device, model.embedding_model, model.embedding_tokenizer, dataset[index]['watermark_text'])
        sim = bias.cosine_similarity(original_embedding, watermark_embedding)
        if sim.item()>=0.95:
            sim_95 += 1
        if 0.9 <= sim.item() < 0.95:
            sim_9 += 1
        sim_total += sim.item()
    average_sim = sim_total / len(dataset)
    result.append({
        'name': name,
        'sim95':sim_95,
        'sim9': sim_9,
        'average': average_sim
    })
    os.makedirs(os.path.dirname(f'text-quality/REPEAT/HC3/result.json'), exist_ok=True)
    with open(f'text-quality/REPEAT/HC3/result.json', 'w', encoding='utf-8') as f:
        json.dump(result, f, indent=4, ensure_ascii=False)
    print(result)

/root/miniconda3/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:818: UserWarning: `return_dict_in_generate` is NOT set to `True`, but `output_hidden_states` is. When `return_dict_in_generate` is not `True`, `output_hidden_states` is ignored.
  warnings.warn(
Some weights of the model checkpoint at /root/autodl-tmp/bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


===bert-base-cased 完毕===


Some weights of the model checkpoint at /root/autodl-tmp/roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


===roberta-large-mnli 完毕===
===roberta-large完毕===
===glove 完毕===
===nltk完毕===


/root/TextRepeat/watermark_REPEAT_NO_CONTEXT_BERT.py:144: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  transform_model.load_state_dict(torch.load(transform_model_path, map_

...

[{'name': 'bloomz', 'sim95': 312, 'sim9': 303, 'average': 0.9257246814668179}, {'name': 'chatGPT', 'sim95': 309, 'sim9': 318, 'average': 0.9270607101917266}, {'name': 'cohere', 'sim95': 348, 'sim9': 288, 'average': 0.92857282795012}, {'name': 'davinci', 'sim95': 308, 'sim9': 311, 'average': 0.9271282063424587}]
===dolly===


100%|██████████| 800/800 [00:17<00:00, 46.05it/s]


[{'name': 'bloomz', 'sim95': 312, 'sim9': 303, 'average': 0.9257246814668179}, {'name': 'chatGPT', 'sim95': 309, 'sim9': 318, 'average': 0.9270607101917266}, {'name': 'cohere', 'sim95': 348, 'sim9': 288, 'average': 0.92857282795012}, {'name': 'davinci', 'sim95': 308, 'sim9': 311, 'average': 0.9271282063424587}, {'name': 'dolly', 'sim95': 488, 'sim9': 234, 'average': 0.9475550780445338}]
===flant5===


100%|██████████| 800/800 [00:16<00:00, 49.58it/s]

[{'name': 'bloomz', 'sim95': 312, 'sim9': 303, 'average': 0.9257246814668179}, {'name': 'chatGPT', 'sim95': 309, 'sim9': 318, 'average': 0.9270607101917266}, {'name': 'cohere', 'sim95': 348, 'sim9': 288, 'average': 0.92857282795012}, {'name': 'davinci', 'sim95': 308, 'sim9': 311, 'average': 0.9271282063424587}, {'name': 'dolly', 'sim95': 488, 'sim9': 234, 'average': 0.9475550780445338}, {'name': 'flant5', 'sim95': 325, 'sim9': 294, 'average': 0.9255265472084284}]
